This file will learn follow reference https://towardsdatascience.com/named-entity-recognition-ner-meeting-industrys-requirement-by-applying-state-of-the-art-deep-698d2b3b4ede

In [2]:
def read_all_line( directory , file ):
    file = open( directory + "/" + file , "r" , encoding="utf-8" )
    contents = file.read()
    return contents

In [14]:
dictionary_directory = "dictionary_directory"
dictionary = {
    "per_front" : read_all_line( dictionary_directory , "clue front.txt").split('\n') +
            read_all_line( dictionary_directory , "person title.txt").split('\n') ,
    "org_front" : read_all_line( dictionary_directory , "คำนำหน้าองค์กรจาก dict.txt").split('\n'),
    "loc" : read_all_line( dictionary_directory , "ชื่อคลอง.txt").split('\n') +
            read_all_line( dictionary_directory , "ชื่อมลรัฐ.txt").split('\n') +
            read_all_line( dictionary_directory , "ชื่อสถานที่.txt").split('\n') +
            read_all_line( dictionary_directory , "ชื่อจังหวัด.txt").split('\n') +
            read_all_line( dictionary_directory , "ชื่อกิ่งอำเภอ.txt").split('\n') +
            read_all_line( dictionary_directory , "ชื่ออำเภอ.txt").split('\n'),
}

In [15]:
def check_dictionary( word ):
    result_have = False
    result_tag = "O"
    for key in dictionary.keys():
        if( word in dictionary[ key ] ):
            result_have = True
            result_tag = key
            break
    return result_have , result_tag

In [18]:
def suffix_processing( word_sequence , result_predict ):
    result = result_predict
    for run in range( len(word_sequence) ):
        have , tag = check_dictionay( word_sequence[ run ] )
        if( have ):
            result[ run ] = tag
            if( tag == "pre_front" ):
                if result[ run + 1 ] == "O":
                    result[ run + 1 ] = "per"
            elif( tag == "org_front" ):
                None
            else:
                None

In [16]:
dictionary

{'per_front': ['จอมพลอากาศ',
  'พลอากาศเอก',
  'พล.อ.อ.',
  'พลอากาศโท',
  'พล.อ.ท.',
  'พลอากาศตรี',
  'พล.อ.ต.',
  'นาวาอากาศเอก',
  'น.อ.',
  'นาวาอากาศโท',
  'น.ท.',
  'นาวาอากาศตรี',
  'น.ต.',
  'เรืออากาศเอก',
  'ร.อ.',
  'เรืออากาศโท',
  'ร.ท.',
  'เรืออากาศตรี',
  'ร.ต.',
  'พันจ่าอากาศเอก',
  'พ.อ.อ.',
  'พันจ่าอากาศโท',
  'พ.อ.ท.',
  'พันจ่าอากาศตรี',
  'พ.อ.ต.',
  'จ่าอากาศเอก',
  'จ.อ.',
  'จ่าอากาศโท',
  'จ.ท.',
  'จ่าอากาศตรี',
  'จ.ต.',
  'พลทหาร',
  'พลฯ',
  'นักเรียนนายเรืออากาศ',
  'นักเรียนจ่าอากาศ',
  'จอมพล',
  'พลเอก',
  'พล.อ.',
  'พลโท',
  'พล.ท.',
  'พลตรี',
  'พล.ต.',
  'พันเอก',
  'พ.อ.',
  'พันโท',
  'พ.ท.',
  'พันตรี',
  'พ.ต.',
  'ร้อยเอก',
  'ร.อ.',
  'ร้อยโท',
  'ร.ท.',
  'ร้อยตรี',
  'ร.ต.',
  'จ่าสิบเอก',
  'จ.ส.อ.',
  'จ่าสิบโท',
  'จ.ส.ท.',
  'จ่าสิบตรี',
  'จ.ส.ต.',
  'สิบเอก',
  'ส.อ.',
  'สิบโท',
  'ส.ท.',
  'สิบตรี',
  'ส.ต.',
  'พลทหาร',
  'พลฯ',
  'นักเรียนนายร้อย',
  'นักเรียนนายสิบทหารบก',
  'จอมพลเรือ',
  'พลเรือเอก',
  'พล.ร.อ.',
  'พลเรือโ

In [6]:
def individual_preprocess( raw_data , split_sentence , count ,remove = [] , use_dictionary = False ):
    pre_data_frame = { "Sentence" : [] , "Word" : [] , "Tag" : [] }
    for word in raw_data:
        have = False
        if use_dictionary:
            have , tag = check_dictionary( word )
            
        if word in remove :
            None
        elif have :
            pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
            pre_data_frame["Word"].append( word )
            pre_data_frame["Tag"].append( tag )
        elif word in split_sentence :
            count += 1
        else: 
            check = word.find('(')
            if check > 0:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word[ 0 : word.find('(')] )
                pre_data_frame["Tag"].append( word[ word.find('(') + 1 : word.find(')')] )
            else:
                pre_data_frame["Sentence"].append( "Sentence: " + str(count) )
                pre_data_frame["Word"].append( word )
                pre_data_frame["Tag"].append( "O" )
    return pre_data_frame , count

In [7]:
import pandas as pd
def preprocess( directory , list_file , count = 1 , individual = False , use_dictionary = False ):
    data_frame = pd.DataFrame( { "Sentence" : [] , "Word" : [] , "Tag" : [] } )
    if( individual ):
        word = read_all_line( directory , list_file ).split('|')
        pre_data_frame , count = individual_preprocess( word ,
                                              ["\n"] ,
                                              count ,
                                              [ "" , '' , '\0'] ,
                                              use_dictionary )
        data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    else:
        for file in list_file :
            word = read_all_line( directory , file ).split('|')
            pre_data_frame , count = individual_preprocess( word ,
                                              ["\n"] ,
                                              count ,
                                              [ "" , '' , '\0']  ,
                                              use_dictionary )
            data_frame  = data_frame.append( pd.DataFrame( pre_data_frame ), ignore_index=True )
    return data_frame

In [8]:
#directory of corpus
corpus_directory = "test_in_class"
corpus_files = ("Politic9.txt" , "Allcolumn.txt" , "98JUL5_1.txt" ,
                "98JUL5_2.txt" , "bkknews1.txt" , "Result1.txt" ,
                "Result2(corpus-1).txt" , "Result3(corpus-2).txt" ,
                "result4.txt" , "Result_c_2_1.txt", "Result_c_2_2.txt" )
corpus_test = ( "POL1108.CUT" , "POL1805.CUT" , "POL1812.CUT" ,  
                "POL1819.CUT" , "POL2008.CUT" , "POL2017.CUT" ,
                "POL1109.CUT" , "POL1806.CUT" , "POL1813.CUT" , 
                "POL2001.CUT" , "POL2009.CUT" , "POL2018.CUT" ,
                "POL1111.CUT" , "POL1807.CUT" , "POL1814.CUT" , 
                "POL2002.CUT" , "POL2010.CUT" , "POL2019.CUT" ,
                "POL1112.CUT" , "POL1808.CUT" , "POL1815.CUT" ,
                "POL2004.CUT" , "POL2011.CUT" , "POL2020.CUT" ,
                "POL1113.CUT" , "POL1809.CUT" , "POL1816.CUT" ,
                "POL2005.CUT" , "POL2012.CUT" , "POL1803.CUT" ,
                "POL1810.CUT" , "POL1817.CUT" , "POL2006.CUT" ,
                "POL2014.CUT" , "POL1804.CUT" , "POL1811.CUT" ,
                "POL1818.CUT" , "POL2007.CUT" , "POL2015.CUT" )
all_corpus = corpus_files + corpus_test
individual = False

In [13]:
data_frame = preprocess( corpus_directory + "/train_data" , 
                        corpus_files, 1 , False , True )
data_frame
number_words = len( set( list( data_frame["Word"].values ) ) )
number_tags = len( list( set( data_frame["Tag"].values ) ) )
print( "Have type of tag is " + str( number_tags ) + 
        " tags from " + str(number_words) + " words")
# print(set( list( data_frame["Tag"].values )))

Have type of tag is 16 tags from 9293 words


In [ ]:
data_frame

Follow https://towardsdatascience.com/named-entity-recognition-and-classification-with-scikit-learn-f05372f07ba2

In [ ]:
data_frame['Sentence'].nunique(), data_frame.Word.nunique(), data_frame.Tag.nunique()

In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
data_frame = data_frame.fillna(method='ffill')
print( "Number of sentence is {:d}".format(data_frame['Sentence'].nunique()) )
print( "Number of word is {:d}".format( data_frame.Word.nunique() ) )
print( "Number of tag is {:d}".format( data_frame.Tag.nunique() ) )

In [ ]:
data_frame.groupby('Tag').size().reset_index(name='counts')

In [ ]:
data_train = data_frame.drop('Tag', axis=1)
#vector_handle = DictVectorizer(sparse=False)
#data_train = vector_handle.fit_transform( data_train.to_dict('records') )
#value_train = data_frame.Tag.values

In [ ]:
test_frame = preprocess( corpus_directory + "/test_tag" , corpus_test , 1 , individual )
#data_test = test_frame.drop('Tag', axis=1)
#vector_handle = DictVectorizer(sparse=False)
#data_test = vector_handle.fit_transform( data_test.to_dict('records') )
#value_test = test_frame.Tag.values

In [ ]:
data_frame.shape[0] , test_frame.shape[0]

In [ ]:
all_frame = data_frame.append( test_frame )
all_data = all_frame.drop('Tag', axis=1)
vector_handle = DictVectorizer(sparse=False)
all_data = vector_handle.fit_transform(all_data.to_dict('records'))
all_value = all_frame.Tag.values
size_of_test = test_frame.shape[0]
print( "Propotional of test is " + str( size_of_test ) )

classes = np.unique( all_value )
classes = classes.tolist()
new_classes = classes.copy()
new_classes.remove('O')
new_classes.remove('')
new_classes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_data, 
        all_value, 
        test_size = size_of_test, 
        shuffle = False )

In [ ]:
# Use Perceptron # the best from three model
per = Perceptron(verbose=10, n_jobs=-1, max_iter=5)
per.partial_fit(X_train, y_train, classes)

In [ ]:
print(classification_report(y_pred=per.predict(X_test), 
                            y_true=y_test, labels=new_classes))

In [ ]:
# Use Passive Aggressive Classifier # Not work
# pa =PassiveAggressiveClassifier()
# pa.partial_fit(X_train, y_train, classes)

In [ ]:
#print(classification_report(y_pred=pa.predict(X_test),
                            #y_true=y_test, labels=new_classes))

In [ ]:
# use naive bayes # NOT WORK
# nb = MultinomialNB(alpha=0.01)
# nb.partial_fit(X_train, y_train, classes)

In [ ]:
#print(classification_report(y_pred=nb.predict(X_test), 
                            #y_true=y_test, labels = new_classes))